# Using ReciPies with Pandas and Polars

This notebook demonstrates how ReciPies works with both Pandas and Polars backends, and how to convert between them during a preprocessing flow.

We'll:
- Build a tiny synthetic dataset
- Run the same `Recipe` with Pandas and with Polars
- Convert between backends while keeping roles and steps consistent


In [ ]:
import numpy as np
import pandas as pd
import polars as pl
from datetime import datetime, timedelta

from recipies import Ingredients, Recipe
from recipies.selector import all_predictors
from recipies.step import StepImputeFill, StepHistorical, Accumulator
from recipies.constants import Backend

rng = np.random.default_rng(42)

# Build a tiny panel/time series dataset
n = 8
ids = np.array([1] * (n // 2) + [2] * (n // 2))
base_time = datetime(2020, 1, 1, 0, 0, 0)
times = np.array([base_time + timedelta(hours=i) for i in range(n)])

x1 = rng.normal(10, 2, size=n)
x2 = rng.integers(0, 2, size=n)
y = rng.normal(0, 1, size=n)

# Inject some missing values
x1[[1, 5]] = np.nan

pdf = pd.DataFrame(
    {
        "id": ids,
        "time": times,
        "x1": x1,
        "x2": x2,
        "y": y,
    }
)

pldf = pl.from_pandas(pdf)

## Use Pandas backend

You can pass a Pandas `DataFrame` directly to `Recipe`. Roles are assigned via constructor arguments.
We impute missing values and compute a rolling historical mean as a simple example.


In [ ]:
rec_pd = Recipe(
    pdf,
    outcomes=["y"],
    predictors=["x1", "x2"],
    groups=["id"],
    sequences=["time"],
)

rec_pd.add_step(StepImputeFill(sel=all_predictors(), strategy="forward"))
rec_pd.add_step(StepHistorical(sel=all_predictors(), fun=Accumulator.MEAN, suffix="mean_hist"))

train_pd = rec_pd.prep()
train_pd.head()

## Switch to Polars

There are two common ways:

1) Start with a Polars `DataFrame` and create your `Recipe` normally.
2) Convert a Pandas `DataFrame` to Polars on-the-fly by constructing `Ingredients` with `backend=Backend.POLARS`.

Both preserve roles and steps.


In [ ]:
# Option 1: Start directly with Polars DataFrame
rec_pl = Recipe(
    pldf,
    outcomes=["y"],
    predictors=["x1", "x2"],
    groups=["id"],
    sequences=["time"],
)
rec_pl.add_step(StepImputeFill(sel=all_predictors(), strategy="forward"))
rec_pl.add_step(StepHistorical(sel=all_predictors(), fun=Accumulator.MEAN, suffix="mean_hist"))
train_pl = rec_pl.prep()
train_pl.head()

In [ ]:
# Option 2: Convert a Pandas DataFrame into Polars via Ingredients
# (This preserves roles and allows you to keep working in Polars.)
ing_pd_to_pl = Ingredients(pdf, backend=Backend.POLARS)
rec_conv = Recipe(
    ing_pd_to_pl,
    outcomes=["y"],
    predictors=["x1", "x2"],
    groups=["id"],
    sequences=["time"],
)
rec_conv.add_step(StepImputeFill(sel=all_predictors(), strategy="forward"))
rec_conv.add_step(StepHistorical(sel=all_predictors(), fun=Accumulator.MEAN, suffix="mean_hist"))
train_conv = rec_conv.prep()
train_conv.head()

## Converting outputs explicitly

If you need to explicitly convert between backends while keeping the data and roles, work with `Ingredients` and `to_df`:

- `Ingredients.to_df(output_format=Backend.PANDAS)`
- `Ingredients.to_df(output_format=Backend.POLARS)`


In [ ]:
# Start with an Ingredients object (Pandas), then convert to Polars and back
ing_pd = Ingredients(pdf, backend=Backend.PANDAS)
# Convert to Polars DataFrame first
pl_df_from_pd = ing_pd.to_df(output_format=Backend.POLARS)
# Create a new Ingredients from the converted Polars DataFrame
ing_pl = Ingredients(pl_df_from_pd, backend=Backend.POLARS)
# Convert back to Pandas
pd_df_from_pl = ing_pl.to_df(output_format=Backend.PANDAS)

pl_df_from_pd.head(), type(pl_df_from_pd), type(pd_df_from_pl)